In [ ]:
!pip uninstall torch torchaudio torchvision -y

Found existing installation: torch 2.3.0
Uninstalling torch-2.3.0:
  Successfully uninstalled torch-2.3.0
Found existing installation: torchaudio 2.2.2
Uninstalling torchaudio-2.2.2:
  Successfully uninstalled torchaudio-2.2.2
Found existing installation: torchvision 0.18.0
Uninstalling torchvision-0.18.0:
  Successfully uninstalled torchvision-0.18.0


In [ ]:
!pip install torch==2.2.2 torchaudio==2.2.2

ERROR: Operation cancelled by user


In [1]:
!pip install torch torchaudio torchvision -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 47.5 MB/s eta 0:00:00


In [2]:
!pip install datasets -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00


In [3]:
import random
import IPython

import datasets
from datasets.utils import DownloadManager


import torch
import torchaudio
import torchaudio.transforms as T

from torch.utils.data import DataLoader, Dataset

import os
import sys

import soundfile as sf

In [4]:
import torch.nn as nn
import torchvision.models as models

In [5]:
from librosa import util
import librosa.feature

In [6]:
import numpy as np

In [7]:
sys.path.append('src')

# Load Dataset .zip from link

Logical access (LA):

- ```speaker_id:``` LA_****, a 4-digit speaker ID
- ```audio_file_name:``` name of the audio file
- ```audio:``` '****.flac'  the path to the downloaded audio file in FLAC format (https://xiph.org/flac/).
- ```system_id:``` ID of the speech spoofing system (A01 - A19), or, for bonafide speech SYSTEM-ID is left blank ('-')
- ```key:``` 'bonafide' for genuine speech, or, 'spoof' for spoofing speech

In [8]:
#This code was taken from the load_avsspoof19.py

import datasets
import os

_HOMEPAGE = "https://datashare.ed.ac.uk/handle/10283/3336"
_URL = "https://datashare.ed.ac.uk/bitstream/handle/10283/3336/LA.zip"
_CITATION = """\
@InProceedings{Todisco2019,
  Title     = {{ASV}spoof 2019: {F}uture {H}orizons in {S}poofed and {F}ake {A}udio {D}etection},
  Author    = {Todisco, Massimiliano and
               Wang, Xin and
               Sahidullah, Md and
               Delgado, H ́ector and
               Nautsch, Andreas and
               Yamagishi, Junichi and
               Evans, Nicholas and
               Kinnunen, Tomi and
               Lee, Kong Aik},
  booktitle = {Proc. of Interspeech 2019},
  Year      = {2019}
}
"""

_DESCRIPTION = """\
This is a database used for the Third Automatic Speaker Verification Spoofing
and Countermeasuers Challenge, for short, ASVspoof 2019 (http://www.asvspoof.org)
organized by Junichi Yamagishi, Massimiliano Todisco, Md Sahidullah, Héctor
Delgado, Xin Wang, Nicholas Evans, Tomi Kinnunen, Kong Aik Lee, Ville Vestman,
and Andreas Nautsch in 2019.
"""

class ASVspoof2019(datasets.GeneratorBasedBuilder):
    BUILDER_CONFIG = datasets.BuilderConfig(name="LA", description="Logical access (LA)")

    def __init__(self):
        super().__init__

    def _info(self):
        # which features to expect in dataset
        print(self.BUILDER_CONFIG.name)
        features = datasets.Features(
            {
                "speaker_id": datasets.Value("string"),
                "audio_file_name": datasets.Value("string"),
                "audio": datasets.Audio(sampling_rate=16_000),
                "system_id": datasets.Value("string"),
                "key": datasets.ClassLabel(names=["bonafide", "spoof"]),
            }
        )
        return datasets.DatasetInfo(
            description=_DESCRIPTION,
            features=features,
            supervised_keys=("audio", "key"),
            homepage=_HOMEPAGE,
            citation=_CITATION,
        )

    def _split_generators(self, dl_manager):
        data_dir = dl_manager.download_and_extract(_URL)
        return [
            datasets.SplitGenerator(
                name=datasets.Split.TRAIN,
                gen_kwargs={
                    "metadata_filepath": os.path.join(
                        data_dir,
                        self.BUILDER_CONFIG.name,
                        f"ASVspoof2019_{self.BUILDER_CONFIG.name}_cm_protocols",
                        f"ASVspoof2019.{self.BUILDER_CONFIG.name}.cm.train.trn.txt",
                    ),
                    "audios_dir": os.path.join(
                        data_dir, self.BUILDER_CONFIG.name, f"ASVspoof2019_{self.BUILDER_CONFIG.name}_train", "flac"
                    ),
                },
            ),
            datasets.SplitGenerator(
                name=datasets.Split.VALIDATION,
                gen_kwargs={
                    "metadata_filepath": os.path.join(
                        data_dir,
                        self.BUILDER_CONFIG.name,
                        f"ASVspoof2019_{self.BUILDER_CONFIG.name}_cm_protocols",
                        f"ASVspoof2019.{self.BUILDER_CONFIG.name}.cm.dev.trl.txt",
                    ),
                    "audios_dir": os.path.join(
                        data_dir, self.BUILDER_CONFIG.name, f"ASVspoof2019_{self.BUILDER_CONFIG.name}_dev", "flac"
                    ),
                },
            ),
            datasets.SplitGenerator(
                name=datasets.Split.TEST,
                gen_kwargs={
                    "metadata_filepath": os.path.join(
                        data_dir,
                        self.BUILDER_CONFIG.name,
                        f"ASVspoof2019_{self.BUILDER_CONFIG.name}_cm_protocols",
                        f"ASVspoof2019.{self.BUILDER_CONFIG.name}.cm.eval.trl.txt",
                    ),
                    "audios_dir": os.path.join(
                        data_dir, self.BUILDER_CONFIG.name, f"ASVspoof2019_{self.BUILDER_CONFIG.name}_eval", "flac"
                    ),
                },
            ),
        ]

    def _generate_examples(self, metadata_filepath, audios_dir):
        data = []
        with open(metadata_filepath) as f:
            for i, line in enumerate(f.readlines()):
                speaker_id, audio_file_name, _, system_id, key = line.strip().split()
                result = {
                    "speaker_id": speaker_id,
                    "audio_file_name": audio_file_name,
                    "system_id": system_id,
                    "key": key,
                }
                result["audio"] = os.path.join(audios_dir, audio_file_name + ".flac")
                data.append(result)
        return data

In [9]:
dl_manager = DownloadManager()

speech_dataset = ASVspoof2019()


asv_datasets = speech_dataset._split_generators(dl_manager)

Computing checksums: 100%|##########| 1/1 [00:42<00:00, 42.33s/it]

In [10]:
# 0 - train, 1 - eval, 2 - test in asv_datasets
train_metadata_filepath = asv_datasets[0].gen_kwargs["metadata_filepath"]
train_audios_dir = asv_datasets[0].gen_kwargs["audios_dir"]

val_metadata_filepath = asv_datasets[1].gen_kwargs["metadata_filepath"]
val_audios_dir = asv_datasets[1].gen_kwargs["audios_dir"]

test_metadata_filepath = asv_datasets[2].gen_kwargs["metadata_filepath"]
test_audios_dir = asv_datasets[2].gen_kwargs["audios_dir"]

In [11]:
train_samples = speech_dataset._generate_examples(train_metadata_filepath, train_audios_dir)
val_samples = speech_dataset._generate_examples(val_metadata_filepath, val_audios_dir)
test_samples = speech_dataset._generate_examples(test_metadata_filepath, test_audios_dir)

## Listen to random sample

Display some audio from train/validation set.

In [13]:
def get_sample(samples):
    return random.sample(samples, k=1)[0]

sample = get_sample(train_samples)
sample

{'speaker_id': 'LA_0089',
 'audio_file_name': 'LA_T_1032079',
 'system_id': 'A05',
 'key': 'spoof',
 'audio': '/root/.cache/huggingface/datasets/downloads/extracted/911103f86670b6f7e96211444d0f39fc5ffab511156a395f67b098c2f45dce18/LA/ASVspoof2019_LA_train/flac/LA_T_1032079.flac'}

In [14]:
# LA_0095 bonatide
print(sample['key'])
IPython.display.Audio(sample['audio'])

spoof


In [15]:
sample = get_sample(val_samples)
sample

{'speaker_id': 'LA_0075',
 'audio_file_name': 'LA_D_3400488',
 'system_id': 'A02',
 'key': 'spoof',
 'audio': '/root/.cache/huggingface/datasets/downloads/extracted/911103f86670b6f7e96211444d0f39fc5ffab511156a395f67b098c2f45dce18/LA/ASVspoof2019_LA_dev/flac/LA_D_3400488.flac'}

In [16]:
# LA_0089 spoof
print(sample['key'])
IPython.display.Audio(sample['audio'])

spoof


## Audio preprocessing


In [17]:
def audio_preprocess(waveform, sample_rate, resample_rate, desired_duration):
    """
        Resample audio to target frequency (16 kHz or 22.05 kHz) \
        Set equal duration for all audios
    """
    resampler = T.Resample(sample_rate, resample_rate)
    resampled_waveform = resampler(waveform)

    desired_length = int(desired_duration * resample_rate)
    if len(resampled_waveform) < desired_length:
        resampled_waveform = resampled_waveform.tile(((desired_length // resampled_waveform.shape[1]) + 1,))
    resampled_waveform = resampled_waveform[:,0: desired_length]

    return resampled_waveform


def peak_normalize(waveform):
    """
        Normalize audio
    """
    waveform /= torch.max(torch.abs(waveform))
    return waveform

In [18]:
class AudioDataset(Dataset):
    def __init__(self, raw_dataset, desired_duration, resample_rate, transform=None):
        self.raw_data = raw_dataset
        self.transform = transform
        self.sample_rate = resample_rate
        self.duration = desired_duration

    def __len__(self):
        return len(self.raw_data)

    def __getitem__(self, idx):
        waveform, sample_rate = torchaudio.load(self.raw_data[idx]['audio'])
        if self.transform:
            waveform = self.transform[0](waveform, sample_rate, self.sample_rate, self.duration)
            waveform = self.transform[1](waveform)

        label = 1 if self.raw_data[idx]['key'] == 'spoof' else 0
        return waveform, label

In [19]:
_DURATION = 6
_SAMPLE_RATE = 16_000

train_dataset = AudioDataset(train_samples, _DURATION, _SAMPLE_RATE, transform=[audio_preprocess, peak_normalize])
train_loader= DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = AudioDataset(test_samples, _DURATION, _SAMPLE_RATE, transform=[audio_preprocess, peak_normalize])
test_loader= DataLoader(test_dataset, batch_size=32, shuffle=True)

In [20]:
train_dataset[0][0].shape

torch.Size([1, 96000])

# Feature extraction  Definition
Mel-frequency cepstrum coefficient (MFCC), constant Q cepstral coefficient (CQCC).

In [21]:
def get_MFCC(waveform, sample_rate):
    mfcc = librosa.feature.mfcc(y=waveform.numpy().squeeze(), sr=sample_rate, n_mfcc=20)
    return torch.from_numpy(mfcc)



In [24]:
def get_LPS(waveform, sample_rate):
    stft = librosa.core.stft(waveform.numpy().squeeze())
    lps = librosa.amplitude_to_db(np.abs(stft))
    return torch.from_numpy(lps)



In [25]:
def get_CQCC(waveform, sample_rate):
    cqcc = librosa.feature.chroma_cqt(y=waveform.numpy().squeeze(), sr=sample_rate)
    return torch.from_numpy(cqcc)





# Model

In [26]:
from tqdm import tqdm
import torchvision

In [27]:
# TODO your code
model = torchvision.models.resnet18(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 119MB/s]


In [28]:
# TODO your code
n_ftrs = model.fc.in_features
model.fc=nn.Linear(n_ftrs,2)
# model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
model.conv1 = nn.Conv2d(1, 64, kernel_size=(9, 3), stride=(3, 1), padding=(1, 1), bias=False)


In [29]:
device= torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(9, 3), stride=(3, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

# Training

In [30]:
num_epochs=3
optimizer=torch.optim.Adam(model.parameters(), lr=0.001)
criterion=nn.CrossEntropyLoss()

In [36]:
def train_loop(model, loader, feature_extractor, criterion, optimizer,n_epochs, device):
    for epoch in range(n_epochs):
        # Training
        model.train(True)
        sum_loss = 0.0
        num_correct = 0
        total_samples = 0
        for data, labels in tqdm(loader):
            data = data
            labels = labels.to(device)

            if feature_extractor == 'MFCC':
                features = get_MFCC(data, loader.dataset.sample_rate)
            elif feature_extractor == 'CQCC':
                features = get_CQCC(data, loader.dataset.sample_rate)
            elif feature_extractor == 'LPS':
                features = get_LPS(data, loader.dataset.sample_rate)
            else:
                raise ValueError(f"Invalid feature extractor: {feature_extractor}")

            features = features.unsqueeze(1).to(device)
            output = model(features)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()


            sum_loss += loss.item()
            max_val, predicted = output.max(1)
            num_correct += (predicted == labels).sum().item()
            total_samples += data.size(0)

        train_accuracy = num_correct / total_samples
        train_avg_loss = sum_loss / len(train_loader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Training Accuracy: {train_accuracy:.4f}, Training Loss: {train_avg_loss:.4f}')


In [ ]:
def evaluate_model(model, test_loader, feature_extractor, criterion, device):
    model.eval()
    test_loss = 0.0
    test_correct = 0
    total_test_samples = 0

    with torch.no_grad():
        for waveform, label in tqdm(test_loader):
            waveform = waveform
            label = label.to(device)

            if feature_extractor == 'MFCC':
                features = get_MFCC(waveform, test_loader.dataset.sample_rate)
            elif feature_extractor == 'CQCC':
                features = get_CQCC(waveform, test_loader.dataset.sample_rate)
            elif feature_extractor == 'LPS':
                features = get_LPS(waveform, test_loader.dataset.sample_rate)
            else:
                raise ValueError(f"Invalid feature extractor: {feature_extractor}")

            features = features.to(device)
            output = model(features)
            loss = criterion(output, label)
            test_loss += loss.item()

            max_val, predicted = output.max(1)
            test_correct += (predicted == label).sum().item()
            total_test_samples += features.size(0)

    test_accuracy = test_correct / total_test_samples
    test_loss = test_loss / len(test_loader)

    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

### Training the model for MFCC features

In [ ]:
feature_extractor='MFCC'

In [ ]:
train_loop(
    model,
    train_loader,
    feature_extractor=feature_extractor,
    criterion=criterion,
    optimizer=optimizer,
    n_epochs=num_epochs,
    device=device
)

100%|██████████| 794/794 [08:51<00:00,  1.49it/s]


accuracy = 0.9925531914893617
avg_loss = 0.023340381654812947


100%|██████████| 794/794 [08:39<00:00,  1.53it/s]


accuracy = 0.9951142631993696
avg_loss = 0.016910217556470993


100%|██████████| 794/794 [08:43<00:00,  1.52it/s]

accuracy = 0.9954294720252167
avg_loss = 0.01600879114641283


In [ ]:
evaluate_model(
    model,
    test_loader,
    feature_extractor=feature_extractor,
    criterion=criterion,
    device=device
)

100%|██████████| 2227/2227 [24:22<00:00,  1.52it/s]

Test Loss: 0.4217, Test Accuracy: 0.8865


### Training the model for LPS features

In [ ]:
feature_extractor='LPS'

In [ ]:
train_loop(
    model,
    train_loader,
    feature_extractor=feature_extractor,
    criterion=criterion,
    optimizer=optimizer,
    n_epochs=num_epochs,
    device=device
)

  0%|          | 0/794 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
100%|██████████| 794/794 [12:10<00:00,  1.09it/s]


Epoch [1/3], Training Accuracy: 0.9938, Training Loss: 0.0189


100%|██████████| 794/794 [11:24<00:00,  1.16it/s]


Epoch [2/3], Training Accuracy: 0.9978, Training Loss: 0.0069


100%|██████████| 794/794 [11:33<00:00,  1.14it/s]

Epoch [3/3], Training Accuracy: 0.9986, Training Loss: 0.0044


In [ ]:
evaluate_model(
    model,
    test_loader,
    feature_extractor=feature_extractor,
    criterion=criterion,
    device=device
)

100%|██████████| 2227/2227 [21:20<00:00,  1.74it/s]

Test Loss: 0.4052, Test Accuracy: 0.8873


### Training the model for CQCC features

In [ ]:
feature_extractor='CQCC'

In [ ]:
train_loop(
    model,
    train_loader,
    feature_extractor=feature_extractor,
    criterion=criterion,
    optimizer=optimizer,
    n_epochs=num_epochs,
    device=device
)

100%|██████████| 794/794 [14:27<00:00,  1.09s/it]


Epoch [1/3], Training Accuracy: 0.9387, Training Loss: 0.1430


  2%|▏         | 16/794 [00:17<14:19,  1.11s/it]

In [ ]:
evaluate_model(
    model,
    test_loader,
    feature_extractor=feature_extractor,
    criterion=criterion,
    device=device
)

100%|██████████| 2227/2227 [21:20<00:00,  1.74it/s]

Test Loss: 0.4052, Test Accuracy: 0.8873


# Model definition

In [ ]:
from models import TE_ResNet

# model = models.TE_ResNet()

# Training

In [ ]:
def train_loop(loader):
    n_epochs = 5
    for epoch in n_epochs:
        for data, labels in loader:
            feature_extr.MFCC() # or any other
            output = model(data)
            loss = ...
    pass